Discussion : Arun
* **Omnidirectional Camera:** Captures images with a wide field of view.
* **OpenCV:** A library used for processing the images captured by the camera.
* **EPnP Model**: Used for determining the camera's position and orientation from captured images.
* **Distortion Model:** Applied to correct the lens distortion inherent in omnidirectional cameras.

### **CloudCompare: Key Uses for 3D Point Clouds**

1. **Visualization**
    - View large point clouds and meshes with advanced display options.
2. **Filtering and Cleaning**
    - Remove noise and outliers, subsample data for efficiency.
3. **Registration**
    - Align multiple point clouds using ICP (Iterative Closest Point).
4. **Segmentation**
    - Divide point clouds based on criteria like color, density, or region.
5. **Measurement and Analysis**
    - Compute distances, calculate volumes, and perform density analysis.
6. **Surface Reconstruction**
    - Create 3D meshes from point cloud data.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture --no-stderr
%pip install -q numpy pandas matplotlib plotly open3d laspy scikit-learn pylas lazrs "laspy[lazrs,laszip]"

In [ ]:
import laspy
import open3d as o3d
import numpy as np

laz_path="/content/drive/MyDrive/2023-08-09_16.02.481-09.08.23-SouthFlare.laz"
las_file = laspy.read(laz_path)

In [ ]:
import numpy as np
import open3d as o3d
import laspy

def random_sample_points(las_file, sample_ratio):
    total_points = len(las_file.points)
    sample_size = int(total_points * sample_ratio)

    # Randomly select indices
    sample_indices = np.random.choice(total_points, sample_size, replace=False)

    # Sample points
    sampled_points = np.vstack((las_file.x[sample_indices], las_file.y[sample_indices], las_file.z[sample_indices])).transpose()

    # Sample colors if available
    if 'red' in las_file.point_format.dimension_names:
        sampled_colors = np.vstack((las_file.red[sample_indices], las_file.green[sample_indices], las_file.blue[sample_indices])).transpose()
        sampled_colors = sampled_colors / 65535.0  # Normalize RGB values to [0, 1]
    else:
        sampled_colors = None

    return sampled_points, sampled_colors

# Load the point cloud data
# las_file = laspy.read('path_to_your_point_cloud_file.las')

# Randomly sample points
sample_ratio = 0.3  # Adjust the ratio as needed
sampled_points, sampled_colors = random_sample_points(las_file, sample_ratio=sample_ratio)

# Create an Open3D point cloud object with the sampled points
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(sampled_points)

# Add colors if available
if sampled_colors is not None:
    point_cloud.colors = o3d.utility.Vector3dVector(sampled_colors)

# Downsample the point cloud for faster processing (optional)
downsampled_pc = point_cloud.voxel_down_sample(voxel_size=1)

# Estimate normals
downsampled_pc.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=0.4, max_nn=30))

# Poisson surface reconstruction
mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(downsampled_pc, depth=8)

# Remove low-density vertices
vertices_to_remove = densities < np.quantile(densities, 0.01)
mesh.remove_vertices_by_mask(vertices_to_remove)

# Simplify mesh if needed
mesh = mesh.simplify_quadric_decimation(50000)

# Remove unwanted artifacts
mesh.remove_degenerate_triangles()
mesh.remove_duplicated_triangles()
mesh.remove_duplicated_vertices()
mesh.remove_non_manifold_edges()

# Save the final mesh to a file
o3d.io.write_triangle_mesh("reconstructed_mesh.ply", mesh)

True

In [ ]:
las=las_file

In [ ]:
x_coordinates = las.x
y_coordinates = las.y
z_coordinates = las.z
print((x_coordinates[0]),(y_coordinates[0]),(z_coordinates[0]))

-173.175095 -689.266174 0.608814


In [ ]:
# Print basic file information
print(f"Total number of points: {len(las.points)}")
print(f"Point format: {las.point_format}")
print(f"Number of returns: {las.num_returns}")

intensity = las.intensity
classification = las.classification

# Return Number and Number of Returns
print("Sample return numbers:", las.return_number[:10])
print("Sample number of returns:", las.number_of_returns[:10])

# RGB Colors (if available)
if 'red' in las.point_format.dimension_names:
    print("Sample RGB values:")
    for i in range(10):
        print(f"Point {i}: (R: {las.red[i]}, G: {las.green[i]}, B: {las.blue[i]})")
else:
    print("RGB data not available.")

# XYZ Coordinates
print("Sample XYZ coordinates:")
for i in range(10):
    print(f"Point {i}: (X: {las.x[i]}, Y: {las.y[i]}, Z: {las.z[i]})")

# Point Density
# Point density can be calculated as the number of points divided by the volume of the bounding box
x_min, x_max = las.x.min(), las.x.max()
y_min, y_max = las.y.min(), las.y.max()
z_min, z_max = las.z.min(), las.z.max()

volume = (x_max - x_min) * (y_max - y_min) * (z_max - z_min)
point_density = len(las.points) / volume if volume > 0 else float('inf')

print(f"Bounding box volume: {volume}")
print(f"Point density: {point_density} points per unit volume")

Total number of points: 169484179
Point format: <PointFormat(2, 0 bytes of extra dims)>
Number of returns: <SubFieldView([1 1 1 ... 1 1 1])>
Sample return numbers: <SubFieldView([1 1 1 1 1 1 1 1 1 1])>
Sample number of returns: <SubFieldView([1 1 1 1 1 1 1 1 1 1])>
Sample RGB values:
Point 0: (R: 28928, G: 25344, B: 22272)
Point 1: (R: 33536, G: 29184, B: 24064)
Point 2: (R: 30976, G: 27136, B: 24320)
Point 3: (R: 32768, G: 28672, B: 26368)
Point 4: (R: 38912, G: 33792, B: 29440)
Point 5: (R: 25088, G: 18944, B: 12288)
Point 6: (R: 29952, G: 25856, B: 22784)
Point 7: (R: 30464, G: 29184, B: 23552)
Point 8: (R: 28416, G: 25600, B: 21504)
Point 9: (R: 35840, G: 28160, B: 22272)
Sample XYZ coordinates:
Point 0: (X: -173.175095, Y: -689.266174, Z: 0.608814)
Point 1: (X: -173.30613699999998, Y: -689.214111, Z: 0.606842)
Point 2: (X: -173.216782, Y: -689.2497559999999, Z: 0.603915)
Point 3: (X: -172.981384, Y: -689.3433229999999, Z: 0.61512)
Point 4: (X: -173.096588, Y: -689.296021, Z: 0.612